In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Rosenbrock

Minimize 2-variable nonsmooth Rosenbrock function, subject to a simple bound constraint. Taken from: [GRANSO](http://www.timmitchell.com/software/GRANSO/) demo examples 1, 2, & 3 

## Problem Description

$$\min_{x_1,x_2} w|x_1^2-x_2|+(1-x_1)^2,$$
$$\text{s.t. }c_1(x_1,x_2) = \sqrt{2}x_1-1 \leq 0, c_(x_1,x_2)=2x_2-1\leq0,$$

where $w$ is a constant (e.g., $w=8$)

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [2]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
device = torch.device('cpu')
# variables and corresponding dimensions.
var_in = {"x1": [1], "x2": [1]}

def comb_fn(X_struct):
    x1 = X_struct.x1
    x2 = X_struct.x2
    
    # objective function
    f = (8 * abs(x1**2 - x2) + (1 - x1)**2)

    # inequality constraint, matrix form
    ci = pygransoStruct()
    ci.c1 = (2**0.5)*x1-1  
    ci.c2 = 2*x2-1 

    # equality constraint 
    ce = None

    return [f,ci,ce]

## User Options
Specify user-defined options for PyGRANSO

In [4]:
opts = pygransoStruct()
# option for switching QP solver. We only have osqp as the only qp solver in current version. Default is osqp
# opts.QPsolver = 'osqp'

# set an intial point
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
opts.x0 = torch.ones((2,1), device=device, dtype=torch.double)
opts.torch_device = device

## Main Algorithm

In [5]:
start = time.time()
cpu_soln = pygranso(var_spec = var_in,combined_fn = comb_fn, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(cpu_soln.final.x)



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## Cuda Computation

In [6]:
device = torch.device('cuda')
# variables and corresponding dimensions.
var_in = {"x1": [1], "x2": [1]}

def comb_fn(X_struct):
    x1 = X_struct.x1
    x2 = X_struct.x2
    
    # objective function
    f = (8 * abs(x1**2 - x2) + (1 - x1)**2)

    # inequality constraint, matrix form
    ci = pygransoStruct()
    ci.c1 = (2**0.5)*x1-1  
    ci.c2 = 2*x2-1 

    # equality constraint 
    ce = None

    return [f,ci,ce]

In [7]:
opts = pygransoStruct()
# option for switching QP solver. We only have osqp as the only qp solver in current version. Default is osqp
# opts.QPsolver = 'osqp'

# set an intial point
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
opts.x0 = torch.ones((2,1), device=device, dtype=torch.double)
opts.torch_device = device

In [8]:
start = time.time()
cuda_soln = pygranso(var_spec = var_in,combined_fn = comb_fn, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(cuda_soln.final.x)



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

In [14]:
cpu_solution = cpu_soln.final.x
cuda_solution = cuda_soln.final.x

torch.testing.assert_close(
    cpu_solution,
    cuda_solution.cpu(),
    atol=1e-5,
    rtol=1e-5,
)